# Análisis de Datos de Producción Eólica
Proyecto de análisis de datos de producción eólica a partir de la API de ESIOS.

---

## 📚 Importación de librerías necesarias

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import plot

## 🔐 Configuración del acceso a la API

In [ ]:
TOKEN = '255c4529289ed8e7cfcfdc5cff2c43d0f101fe5b3adaa20273c01b0deafa80d4'
HEADERS = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'x-api-key': TOKEN,
    'User-Agent': 'esios-api-client'
}

## 📅 Selección de fechas (entrada manual)

In [ ]:
while True:
    try:
        start = datetime.strptime(input("Introduce la fecha de inicio (dd/mm/yyyy): "), "%d/%m/%Y")
        end = datetime.strptime(input("Introduce la fecha de fin (dd/mm/yyyy): "), "%d/%m/%Y")
        if start > end:
            print("⚠️ La fecha de inicio debe ser anterior a la fecha de fin.")
            continue
        break
    except ValueError:
        print("⚠️ Formato no válido. Usa dd/mm/yyyy.")

## 📥 Función para obtener datos desde la API de ESIOS

In [ ]:
def get_esios_data(indicator_id, start_date, end_date):
    url = f'https://api.esios.ree.es/indicators/{indicator_id}'
    params = {
        'start_date': start_date.isoformat(),
        'end_date': end_date.isoformat(),
        'time_trunc': 'hour'
    }
    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code == 200:
        data = response.json()
        values = data['indicator']['values']
        df = pd.DataFrame(values)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df[['datetime', 'value']].rename(columns={'value': f'indicator_{indicator_id}'})
        return df
    else:
        print(f"Error {response.status_code}: {response.text}")
        return pd.DataFrame(columns=['datetime', f'indicator_{indicator_id}'])

## 📊 Descarga de datos de previsión y producción real

In [ ]:
df_forecast = get_esios_data(541, start, end)
df_real = get_esios_data(551, start, end)
df = pd.merge(df_forecast, df_real, on='datetime', how='outer').sort_values('datetime')

## 🧹 Limpieza de datos

In [ ]:
def cleaning(df):
    df = df.copy()
    df.set_index('datetime', inplace=True)
    df.interpolate('linear', inplace=True)
    return df.reset_index()

df = cleaning(df)

for col in ['indicator_541', 'indicator_551']:
    Q1, Q3 = df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    df[col] = df[col].where(df[col].between(0, Q3 + 1.5 * IQR))

df = cleaning(df)
df['datetime'] = df['datetime'].dt.tz_localize(None)
df.loc[df['datetime'] > datetime.now(), 'indicator_551'] = 0
df['indicator_551'] /= 12
df['indicator_541'] /= 4

## 💾 Exportación de datos limpios a Excel

In [ ]:
df.to_excel("WIND_DATAv2.xlsx", index=False)
print("Datos exportados correctamente a 'WIND_DATAv2.xlsx'")

## 📈 Análisis de correlación entre previsión y producción real

In [ ]:
df_corr = df.dropna(subset=['indicator_541', 'indicator_551'])
correlation = df_corr['indicator_541'].corr(df_corr['indicator_551'])
print("Coeficiente de correlación:", correlation)

plt.figure(figsize=(10, 6))
sns.regplot(data=df_corr, x='indicator_541', y='indicator_551',
            line_kws={'color': 'red'}, scatter_kws={'alpha': 0.5})
plt.title(f'Correlación previsión vs real (r = {correlation:.2f})')
plt.xlabel('Previsión eólica (MW)')
plt.ylabel('Producción real (MW)')
plt.grid(True)
plt.tight_layout()
plt.show()

m, b = np.polyfit(df_corr['indicator_541'], df_corr['indicator_551'], 1)

## 📆 Agrupación diaria y cálculo de error

In [ ]:
df['date'] = df['datetime'].dt.date
df_daily = df.groupby('date', as_index=False)[['indicator_541', 'indicator_551']].sum()
df_daily['error'] = df_daily['indicator_541'] - df_daily['indicator_551']

## 📊 Comparación diaria de previsión y producción real (barras)

In [ ]:
x = np.arange(len(df_daily['date']))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, df_daily['indicator_541'], width, label='Previsión diaria', color='skyblue')
plt.bar(x + width/2, df_daily['indicator_551'], width, label='Producción real diaria', color='lightgreen')
plt.xticks(x, df_daily['date'], rotation=45)
plt.xlabel('Fecha')
plt.ylabel('Energía diaria total (MW·h)')
plt.title('Comparación diaria de energía: Previsión vs Producción real')
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

## 🔄 Comparativa con datos del Grupo 1 (Zona 7 - Castilla-La Mancha)

In [ ]:
def get_generacion_zona7(start, end):
    url = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion"
    headers = {"Accept": "application/json"}
    params = {
        "start_date": start.strftime("%Y-%m-%dT%H:%M"),
        "end_date": end.strftime("%Y-%m-%dT%H:%M"),
        "time_trunc": "day",
        "geo_limit":  "ccaa",
        "geo_id": 7
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"⛔ Error {response.status_code}")
        return pd.DataFrame()

    data = response.json()
    rows = []
    for tech in data["included"]:
        if tech["attributes"]["title"] == "Eólica":
            for v in tech["attributes"]["values"]:
                rows.append({
                    "date": v["datetime"][:10],
                    "eolica_zona7": v["value"]
                })
    return pd.DataFrame(rows)

df_zona7 = get_generacion_zona7(start, end)
df_zona7['date'] = pd.to_datetime(df_zona7['date'])
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_comparado = pd.merge(df_daily, df_zona7, on='date', how='inner')

## 📉 Comparativa visual con datos del Grupo 1

In [ ]:
df_melted = df_comparado.melt(id_vars="date", 
                              value_vars=["indicator_541", "indicator_551", "eolica_zona7"],
                              var_name="Fuente", 
                              value_name="Energía (MWh)")

nombre_dict = {
    "indicator_541": "Previsión (Grupo 2 - ESIOS)",
    "indicator_551": "Producción real (Grupo 2 - ESIOS)",
    "eolica_zona7": "Generación eólica real Zona 7 (Grupo 1)"
}
df_melted["Fuente"] = df_melted["Fuente"].map(nombre_dict)

sns.lineplot(data=df_melted, x="date", y="Energía (MWh)", hue="Fuente", marker="o")
plt.title("Comparación diaria de generación eólica")
plt.xlabel("Fecha")
plt.ylabel("Energía (MWh)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

## 📊 Dashboard interactivo en Jupyter

In [ ]:
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Forecast vs Real Power over Time",
        "Forecast vs Real Correlation (scatter)",
        "Comparación diaria generación eólica (Grupo 2 vs Grupo 1)",
        "Error diario de previsión"
    )
)

fig_dashboard.add_trace(
    go.Scatter(x=df_corr['datetime'], y=df_corr['indicator_541'], mode='lines', name='Forecast Power', line=dict(color='blue')),
    row=1, col=1
)
fig_dashboard.add_trace(
    go.Scatter(x=df_corr['datetime'], y=df_corr['indicator_551'], mode='lines', name='Real Power', line=dict(color='green')),
    row=1, col=1
)

fig_dashboard.add_trace(
    go.Scatter(x=df_corr['indicator_541'], y=df_corr['indicator_551'], mode='markers', name='Data Points', marker=dict(color='dodgerblue')),
    row=1, col=2
)
m, b = np.polyfit(df_corr['indicator_541'], df_corr['indicator_551'], 1)
fig_dashboard.add_trace(
    go.Scatter(x=df_corr['indicator_541'], y=m * df_corr['indicator_541'] + b, mode='lines', name='Trend Line', line=dict(color='firebrick', dash='dash', width=2)),
    row=1, col=2
)

fig_dashboard.add_trace(
    go.Scatter(x=df_comparado['date'], y=df_comparado['indicator_541'], mode='lines+markers', name='Previsión diaria (Grupo 2)', line=dict(color='dodgerblue')),
    row=2, col=1
)
fig_dashboard.add_trace(
    go.Scatter(x=df_comparado['date'], y=df_comparado['indicator_551'], mode='lines+markers', name='Producción real diaria (Grupo 2)', line=dict(color='mediumseagreen')),
    row=2, col=1
)
fig_dashboard.add_trace(
    go.Scatter(x=df_comparado['date'], y=df_comparado['eolica_zona7'], mode='lines+markers', name='Generación eólica Zona 7 (Grupo 1)', line=dict(color='orange')),
    row=2, col=1
)

fig_dashboard.add_trace(
    go.Bar(x=df_daily['date'], y=df_daily['error'], marker_color='indianred', name='Error diario'),
    row=2, col=2
)

fig_dashboard.update_layout(
    height=900, width=1200,
    title_text="Dashboard Comparativo Energía Eólica",
    showlegend=True,
    template="plotly_white"
)

fig_dashboard.show()